Natural Scenes Dataset (NSD) 

https://cvnlab.slite.page/p/CT9Fwl4_hc/NSD-Data-Manual 

In [0]:
%pip install h5py fsspec requests aiohttp

In [0]:
## STIMULI
import pandas as pd


path = 'https://natural-scenes-dataset.s3.amazonaws.com/nsddata/ppdata/subj01/behav/responses.tsv'
target_df = pd.read_csv(path,
                 delimiter='\t')
# can the brain activity predict decision correctness (i.e. was it a recent image?)? target ==> ISCORRECT
# can the brain activity predict image (i.e. was it a seen image from whole session?)? target ==> ISOLD 
# can the brain activity predict image from current session (i.e. was it a seen image from current session?)? target ==> ISOLDCURRENT
target_df

In [0]:
import h5py
import fsspec

URL = 'https://natural-scenes-dataset.s3.amazonaws.com/nsddata_betas/ppdata/subj01/func1mm/betas_fithrf_GLMdenoise_RR/betas_session01.hdf5' # Assuming a publicly accessible url

remote_f = fsspec.open(URL, mode="rb")
if hasattr(remote_f, "open"):
    remote_f = remote_f.open()

f = h5py.File(remote_f)

In [0]:
f['betas']

In [0]:
import numpy as np
trial = 20
stack = 85
# trial, stack, y, x
# We divide by 300 to get it in percent signal change. Original data is multipled by 300 to ensure data has high enough dynamic range to cover. 
# get the fMRI signals for a single stack across all the trials.
img = f['betas'][:, stack, :, :].astype('float32')/300

In [0]:
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap

cmapsign4 = np.array([
  [.8, 1, 1], # cyan-white
  [0, 1, 1],  # cyan
  [0, 0, 1],  # blue
  [0, 0, 0],  # black
  [1, 0, 0],  # red
  [1, 1, 0],  # yellow
  [1, 1, .8], # yellow-white
  ])
custom_cmap = ListedColormap(cmapsign4)
p1, p99 = np.percentile(img.mean(axis=0), [1, 99])

rotated_img = np.rot90(img.mean(axis=0))
plt.imshow(rotated_img, cmap=custom_cmap, vmin=-p99, vmax=p99)
plt.colorbar() # Display the image with Berlin colormap
plt.show()

# ETL images all into a Delta Table.
We recommend to paths when working with image datasets in Databricks:
1. Create a Delta Table with the image stashed directly into the table. This is the preferred route for smaller images (e.g. single channel)
2. Stash the images into Databricks Volumes and then create a Delta Table that has a pointer to each image. 

Since our images are single-channel (i.e. percent change) and 186x145px for 1mm resolution or 104x81 for 1.8mm resolution, we will follow option 1 and stash the image directly into a Delta Table. 

In [0]:
SUBJECT = '01' #01:08
SESSION = '01' #01:37
RESOLUTION = "1mm" #can be '1mm' or '1pt8mm'

# Natural Scenes Dataset (NSD) publicly accessible URL for fMRI images 
response_url = f'https://natural-scenes-dataset.s3.amazonaws.com/nsddata_betas/ppdata/subj{SUBJECT}/func{RESOLUTION}/betas_fithrf_GLMdenoise_RR/betas_session{SESSION}.hdf5' 
behavioral_url = f'https://natural-scenes-dataset.s3.amazonaws.com/nsddata/ppdata/subj{SUBJECT}/behav/responses.tsv'

In [0]:
# Open behavior TSV file
behavioral_df = pd.read_csv(behavioral_url, delimiter='\t')

# Open fMRI hdf5 file
remote_f = fsspec.open(URL, mode="rb")
if hasattr(remote_f, "open"):
    remote_f = remote_f.open()

response_dict = h5py.File(remote_f)['betas']

In [0]:
target_df

In [0]:
pd.options.mode.chained_assignment = None

# Get behavioral data for this specific session
session_behavioral_df = behavioral_df.loc[(behavioral_df['SESSION'] == int(SESSION))].reset_index(drop=True)

num_trials = response_dict.shape[0]
num_stacks = response_dict.shape[1]

final_df = pd.DataFrame()

for trial in range(0, num_trials):
  for stack in range(0, num_stacks):
    print(trial, stack)
    # We divide by 300 to get it in percent signal change. Original data is multipled by 300 to ensure data has high enough dynamic range to cover. 
    img = response_dict[trial, stack, :, :].astype('float32')/300
    
    # data to append
    write = session_behavioral_df.iloc[trial]
    write['IMAGE_SLICE'] = stack
    write['IMAGE'] = img
    write = pd.DataFrame(write).transpose()

    final_df = pd.concat([final_df, write])




In [0]:
final_df